# Postprocessing
Load matched disaster datasets, clean, and merge if matched with same GFD floods.

In [118]:
# Import modules
import os
import numpy as np
import pandas as pd 
import geopandas as gpd
from datetime import datetime
import itertools
import glob
          
# Create paths

def mkdir(dir):    
    if not os.path.exists(dir):
        os.mkdir(dir)
        
path_local = os.getcwd() + '/'
path_data = path_local + 'data/'
path_data_processed = path_local + 'data_processed/'
path_FLODIS_final = path_data_processed + 'FLODIS_final/'
path_FLODIS_final_creator = mkdir(path_FLODIS_final)

# NatCat functions

In [121]:
# Delete unneccessary rows and columns. 
# Check for national vs. subnational entries.
# Search for entries with the same GFD ID. 

def drop_columns_rows(df):
    
    df = df[df.columns.drop(df.filter(regex='Unnamed:'))]     
    df = df.drop([
        
'begin', 'end',
       'event', 'type', 'subtype', 
        'country', 'Continent', 
      
      'Longitude', 'Latitude', 'Reg_name', 'ID', 'Reg_ID',
    
    ],axis=1)
    df = df.replace(-np.inf,np.nan)
    
    return df

def matches_filter(df):
    print('Number of NatCat flood events (2000-2016):',len(df))
    df = df[df.DFO_matches > 0]
    print('Number of succesfully matched NatCat events:',len(df),'\n\n')
    df = df.reset_index(drop=True)
    
    return df

def NatCat_duplicates(df):

    sum_columns = ['tot_loss',
       'ins_loss', 'tot_loss_GDP', 'tot_loss_GCP', 'tot_loss_CPI',
       'Fatalities', 
       'CPI_conv_ppp', 'GDP_conv_ppp',]

    mean_columns = ['DFO_matches_time_dif', 'DFO_duration',        
           'DFO_satellite_shape_pop_density_GHSL', 
            'DFO_satellite_shape_pop_density_GPW','affected_mean_GHSL','affected_mean_GPW','GDP_affected_mean',
           'CISI_global_affected_mean', 'cable_affected_mean',
           'plant_affected_mean', 'power_pole_affected_mean',
           'power_tower_affected_mean', 'line_affected_mean', 'mast_affected_mean',
           'communication_tower_affected_mean', 'doctors_affected_mean',
           'hospital_affected_mean', 'pharmacy_affected_mean',
           'primary_road_affected_mean', 'tertiary_road_affected_mean',
           'reservoir_affected_mean', 'school_affected_mean',
           'university_affected_mean', 'GDPpc_mean', 'HDI_mean',
           'urbanization_mean', 'landuse_total_mean', 'female_mean',               
           'pop_0_14_mean', 'pop_65_plus_mean', 'FLOPROS_merged_mean',
           'FLOPROS_modeled_mean', 'forest_cover_mean']

    max_columns = ['DFO_satellite_shape_area','affected_sum_GHSL', 
                   'affected_sum_GPW','GDP_affected_sum', 'cable_affected_sum', 'plant_affected_sum',
                   'power_pole_affected_sum', 'power_tower_affected_sum',
                   'line_affected_sum', 'mast_affected_sum',
                   'communication_tower_affected_sum', 'doctors_affected_sum',
                   'hospital_affected_sum', 'pharmacy_affected_sum',
                   'primary_road_affected_sum', 'tertiary_road_affected_sum',
                   'reservoir_affected_sum', 'school_affected_sum',
                   'university_affected_sum']
    
    for ID in df.DFO_matches_nr.unique():

        df_temp_1 = df[df['DFO_matches_nr']==ID]

        for ISO in df_temp_1.ISO.unique():
            
         #   print(ISO)

            df_temp_2 = df_temp_1[df_temp_1.ISO == ISO]

            if len(df_temp_2) > 1: 
                
          #      df_temp_2.to_csv('del.csv')
                
                print('nat. entry > subnat. entry')
              #  print(ISO)
               # print(ID)
                print(df_temp_2.begin_dt)
                print(df_temp_2.begin_dt.mean())
                
                ISO3_temp = df_temp_2.ISO.iloc[0]
                datetime_temp = pd.to_datetime(df_temp_2.begin_dt).mean()
           #     print(ISO3_temp)
                DFO_matches_temp = df_temp_2.DFO_matches.max()
                DFO_matching_type_temp = 3
            #    print(df_temp_2.DFO_matches_nr)
             #   print(len(df_temp_2.DFO_matches_nr))

                DFO_matches_nr_temp = ID
             #   print("New nr:",DFO_matches_nr_temp)
#                 disasterno_temp = ''
                
#                 for i in df_temp_2.disasterno:

#                     if str(i) != 'nan':
#                         disasterno_temp = disasterno_temp + str(i) + ','

#                 try:
#                     if disasterno_temp[-1] == ',':
#                         disasterno_temp = disasterno_temp[:-1]        
#                 except:
#                     pass

                df_max = pd.DataFrame(df_temp_2[max_columns].max()).T
                df_mean = pd.DataFrame(df_temp_2[mean_columns].mean()).T
                df_sum = pd.DataFrame(df_temp_2[sum_columns].sum()).T
                row_single = pd.concat([df_sum,df_max,df_mean], axis=1)

                try:
                    row_single = row_single.drop(['disasterno'],axis=1)
                except:
                    pass

                row_single.insert(0,'ISO',ISO3_temp)
                row_single.insert(1,'DFO_matches',DFO_matches_temp)
                row_single.insert(2,'matching_type',DFO_matching_type_temp)
                row_single.insert(3,'DFO_matches_nr',DFO_matches_nr_temp)
                row_single.insert(4,'begin_dt',datetime_temp)
#                 row_single.insert(4,'disasterno',disasterno_temp)
                row_single['year'] = np.round(df_temp_2.year.mean(),0)
                row_single = row_single.reindex(columns=df.columns)
                df = df.drop(df_temp_2.index.to_list())
                df = df.append(row_single)
                
                print(row_single.begin_dt)

             #   print(row_single.DFO_matches_nr)
                print(row_single.ISO)
             
    print('Number of succesfully matched Nat-Cat events:',len(df))
    print("Number of individual DFO events:",len(df.DFO_matches_nr.unique()),'\n\n')
    df = df.reset_index(drop=True)    
    
    return(df)

# Launch (Nat-Cat)

In [122]:
df = pd.read_csv(path_data_processed + "IDMC_EMDAT_GFD_match/NatCat_GFD_connect_results_FL_merged.csv")

def datetime_conversion(date_temp):
   # print(date_temp)
    if len(str(date_temp)) <= 10:
        begin_dt = datetime.strptime(date_temp, "%Y-%m-%d")
    else:
        begin_dt = datetime.strptime(date_temp, "%Y-%m-%d %H:%M:%S")
        
    return(begin_dt)

df['begin_dt'] = df.begin.apply(lambda x: datetime_conversion(x))

# df.rename(columns={'iso3':'ISO3','Total Deat':'total_deaths', 'No Injured':'no_injured', 
#                    'No Affecte':'no_affected_EMDAT', 'No Homeles':'no_homeles','Total Affe':'total_affected_EMDAT',
#                    'Total Dama':'total_damages_(000_USD)',}, 
#                      inplace=True)  

for i in range(len(df)):    
    ID = df.loc[i,'DFO_matches_nr']
    
    if "[" in str(ID) and len(ID) <= 6:        
        df.loc[i,'DFO_matches_nr'] = ID.replace('[','').replace(']','')
 
df = (df.
                pipe(drop_columns_rows).
                pipe(matches_filter).
                pipe(NatCat_duplicates)
             )

print("FINAL:")
print('Number of succesfully matched Nat-Cat events:',len(df))
print('Number of events without fatalities and damages numbers:',len(df[df['tot_loss'].isnull() & df.Fatalities.isnull()]))

df.to_csv(path_FLODIS_final + 'FLODIS_NatCat_fatalities_damages.csv')

Number of NatCat flood events (2000-2016): 5642
Number of succesfully matched NatCat events: 1100 


nat. entry > subnat. entry
0   2000-03-09
1   2000-04-06
Name: begin_dt, dtype: datetime64[ns]
2000-03-23 00:00:00
0   2000-03-23
Name: begin_dt, dtype: datetime64[ns]
0    ROU
Name: ISO, dtype: object
nat. entry > subnat. entry
11   2001-05-07
13   2001-05-28
Name: begin_dt, dtype: datetime64[ns]
2001-05-17 12:00:00
0   2001-05-17 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    RUS
Name: ISO, dtype: object
nat. entry > subnat. entry
18   2001-08-20
21   2001-09-01
Name: begin_dt, dtype: datetime64[ns]
2001-08-26 00:00:00
0   2001-08-26
Name: begin_dt, dtype: datetime64[ns]
0    VNM
Name: ISO, dtype: object
nat. entry > subnat. entry
33   2002-04-19
36   2002-04-28
Name: begin_dt, dtype: datetime64[ns]
2002-04-23 12:00:00
0   2002-04-23 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    BGD
Name: ISO, dtype: object
nat. entry > subnat. entry
42   2002-05-22
48   2002-06-13
55   200

0   2007-10-21 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    VNM
Name: ISO, dtype: object
nat. entry > subnat. entry
460   2007-11-18
470   2008-01-01
Name: begin_dt, dtype: datetime64[ns]
2007-12-10 00:00:00
0   2007-12-10
Name: begin_dt, dtype: datetime64[ns]
0    MWI
Name: ISO, dtype: object
nat. entry > subnat. entry
468   2007-12-25
471   2008-01-01
Name: begin_dt, dtype: datetime64[ns]
2007-12-28 12:00:00
0   2007-12-28 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    IDN
Name: ISO, dtype: object
nat. entry > subnat. entry
477   2008-02-01
480   2008-02-08
Name: begin_dt, dtype: datetime64[ns]
2008-02-04 12:00:00
0   2008-02-04 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    IDN
Name: ISO, dtype: object
nat. entry > subnat. entry
481   2008-02-11
483   2008-02-24
Name: begin_dt, dtype: datetime64[ns]
2008-02-17 12:00:00
0   2008-02-17 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    AUS
Name: ISO, dtype: object
nat. entry > subnat. entry
491   2008-06-07
496   200

0   2011-10-03 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    GTM
Name: ISO, dtype: object
nat. entry > subnat. entry
743   2011-09-30
748   2011-10-11
Name: begin_dt, dtype: datetime64[ns]
2011-10-05 12:00:00
0   2011-10-05 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    HND
Name: ISO, dtype: object
nat. entry > subnat. entry
756   2011-11-21
758   2011-11-27
Name: begin_dt, dtype: datetime64[ns]
2011-11-24 00:00:00
0   2011-11-24
Name: begin_dt, dtype: datetime64[ns]
0    ZAF
Name: ISO, dtype: object
nat. entry > subnat. entry
759   2011-11-27
765   2011-12-21
Name: begin_dt, dtype: datetime64[ns]
2011-12-09 00:00:00
0   2011-12-09
Name: begin_dt, dtype: datetime64[ns]
0    TZA
Name: ISO, dtype: object
nat. entry > subnat. entry
760   2011-12-03
763   2011-12-18
768   2011-12-25
770   2012-01-01
Name: begin_dt, dtype: datetime64[ns]
2011-12-19 12:00:00
0   2011-12-19 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    IDN
Name: ISO, dtype: object
nat. entry > subnat. entry
7

0   2015-10-29
Name: begin_dt, dtype: datetime64[ns]
0    PHL
Name: ISO, dtype: object
nat. entry > subnat. entry
1001   2015-11-08
1003   2015-11-13
1008   2015-12-04
1015   2015-12-24
1018   2015-12-29
Name: begin_dt, dtype: datetime64[ns]
2015-12-03 14:24:00
0   2015-12-03 14:24:00
Name: begin_dt, dtype: datetime64[ns]
0    GBR
Name: ISO, dtype: object
nat. entry > subnat. entry
1009   2015-12-04
1016   2015-12-28
Name: begin_dt, dtype: datetime64[ns]
2015-12-16 00:00:00
0   2015-12-16
Name: begin_dt, dtype: datetime64[ns]
0    IRL
Name: ISO, dtype: object
nat. entry > subnat. entry
1005   2015-11-17
1006   2015-12-01
Name: begin_dt, dtype: datetime64[ns]
2015-11-24 00:00:00
0   2015-11-24
Name: begin_dt, dtype: datetime64[ns]
0    PRY
Name: ISO, dtype: object
nat. entry > subnat. entry
1019   2016-01-03
1025   2016-01-14
Name: begin_dt, dtype: datetime64[ns]
2016-01-08 12:00:00
0   2016-01-08 12:00:00
Name: begin_dt, dtype: datetime64[ns]
0    TZA
Name: ISO, dtype: object
nat. entr

In [125]:
df[df.begin_dt.isnull()]

,year,ISO,tot_loss,ins_loss,tot_loss_GDP,tot_loss_GCP,tot_loss_CPI,Fatalities,CPI_conv,GDP_conv,...,elevation,roughness,slope,female_mean,pop_0_14_mean,pop_65_plus_mean,FLOPROS_merged_mean,FLOPROS_modeled_mean,forest_cover_mean,begin_dt


In [114]:
test.begin_dt

0    2001-05-07 00:00:00
1    2001-05-28 00:00:00
Name: begin_dt, dtype: object

Timestamp('2001-05-17 12:00:00')

In [103]:
test = pd.read_csv('del.csv')

test.begin_dt#.mean()

0    2001-05-07 00:00:00
1    2001-05-28 00:00:00
Name: begin_dt, dtype: object

In [83]:
df.begin_dt[700:740]

700    2016-04-01 00:00:00
701    2016-04-02 00:00:00
702    2016-04-08 00:00:00
703    2016-04-15 00:00:00
704    2016-05-02 00:00:00
705    2016-05-26 00:00:00
706    2016-06-09 00:00:00
707    2016-06-21 00:00:00
708    2016-07-18 00:00:00
709    2016-07-19 00:00:00
710    2016-07-21 00:00:00
711    2016-07-25 00:00:00
712    2016-07-30 00:00:00
713    2016-08-11 00:00:00
714    2016-10-20 00:00:00
715    2016-12-01 00:00:00
716    2016-12-24 00:00:00
717    2016-12-25 00:00:00
718    2016-12-25 00:00:00
719                    NaN
720                    NaN
721                    NaN
722                    NaN
723                    NaN
724                    NaN
725                    NaN
726                    NaN
727                    NaN
728                    NaN
729                    NaN
730                    NaN
731                    NaN
732                    NaN
733                    NaN
734                    NaN
735                    NaN
736                    NaN
7

In [77]:
df = pd.read_csv(path_data_processed + "IDMC_EMDAT_GFD_match/NatCat_GFD_connect_results_FL_merged.csv")

def datetime_conversion(date_temp):
   # print(date_temp)
    if len(str(date_temp)) <= 10:
        begin_dt = datetime.strptime(date_temp, "%Y-%m-%d")
    else:
        begin_dt = datetime.strptime(date_temp, "%Y-%m-%d %H:%M:%S")
        
    return(begin_dt)

df['begin_dt'] = df.begin.apply(lambda x: datetime_conversion(x))

df[df.year == 2016].begin_dt.mean()

Timestamp('2016-07-04 11:31:17.740864')

In [32]:
len(str(df.loc[0,'begin']))

10

2000-01-01
2000-01-05
2000-01-12
2000-01-13
2000-01-14
2000-01-17
2000-01-26
2000-01-31
2000-02-01
2000-02-04
2000-02-05
2000-02-05
2000-02-05
2000-02-05
2000-02-05
2000-02-05
2000-02-06
2000-02-07
2000-02-09
2000-02-17
2000-02-19
2000-02-28
2000-03-09
2000-03-09
2000-03-09
2000-03-12
2000-03-13
2000-03-13
2000-03-13
2000-03-21
2000-03-21
2000-03-21
2000-03-25
2000-03-31
2000-04-01
2000-04-01
2000-04-01
2000-04-06
2000-04-06
2000-04-06
2000-04-08
2000-04-19
2000-04-19
2000-04-21
2000-04-22
2000-04-26
2000-05-01
2000-05-02
2000-05-08
2000-05-09
2000-05-11
2000-05-12
2000-05-13
2000-05-14
2000-05-15
2000-05-16
2000-05-16
2000-05-17
2000-05-17
2000-05-20
2000-05-22
2000-05-24
2000-05-26
2000-05-27
2000-05-27
2000-05-28
2000-05-30
2000-05-31
2000-06-01
2000-06-04
2000-06-06
2000-06-10
2000-06-10
2000-06-11
2000-06-12
2000-06-12
2000-06-12
2000-06-13
2000-06-13
2000-06-13
2000-06-13
2000-06-14
2000-06-20
2000-06-20
2000-06-21
2000-06-21
2000-06-22
2000-06-24
2000-06-25
2000-06-25
2000-06-26

2014-02-10 00:00:00
2014-02-10 00:00:00
2014-02-10 00:00:00
2014-02-10 00:00:00
2014-02-11 00:00:00
2014-02-12 00:00:00
2014-02-12 00:00:00
2014-02-13 00:00:00
2014-02-13 00:00:00
2014-02-13 00:00:00
2014-02-14 00:00:00
2014-02-16 00:00:00
2014-02-16 00:00:00
2014-02-18 00:00:00
2014-02-22 00:00:00
2014-02-22 00:00:00
2014-02-22 00:00:00
2014-02-22 00:00:00
2014-02-24 00:00:00
2014-02-27 00:00:00
2014-02-28 00:00:00
2014-03-01 00:00:00
2014-03-01 00:00:00
2014-03-01 00:00:00
2014-03-01 00:00:00
2014-03-03 00:00:00
2014-03-03 00:00:00
2014-03-03 00:00:00
2014-03-04 00:00:00
2014-03-04 00:00:00
2014-03-04 00:00:00
2014-03-06 00:00:00
2014-03-06 00:00:00
2014-03-07 00:00:00
2014-03-10 00:00:00
2014-03-10 00:00:00
2014-03-11 00:00:00
2014-03-12 00:00:00
2014-03-13 00:00:00
2014-03-14 00:00:00
2014-03-14 00:00:00
2014-03-14 00:00:00
2014-03-14 00:00:00
2014-03-14 00:00:00
2014-03-15 00:00:00
2014-03-17 00:00:00
2014-03-20 00:00:00
2014-03-21 00:00:00
2014-03-23 00:00:00
2014-03-24 00:00:00


In [40]:
df['begin_dt'] = df.begin.apply(lambda x: print(datetime_conversion(x)))

2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00


2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00


2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00
2000-01-01 00:00:00


In [50]:
df.begin_dt

0      2000-01-01
1      2000-01-05
2      2000-01-12
3      2000-01-13
4      2000-01-14
          ...    
5637   2016-12-26
5638   2016-12-27
5639   2016-12-28
5640   2016-12-29
5641   2016-12-30
Name: begin_dt, Length: 5642, dtype: datetime64[ns]